<a href="https://colab.research.google.com/github/sim273/valifact/blob/master/doc_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import nltk
nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv
import pandas as pd

e=[]
tokenize = lambda doc: doc.lower().split(" ")
df1=pd.read_csv("stories/Book1.csv")
for index,row in df1.iterrows():
  #print(row["SOURCE"],row["CLAIM"])
  file_name = 'stories/'
  file_name  += row['SOURCE'] + ".txt"
  with open(file_name, 'r+', encoding="utf-8") as f:
    s2 = f.read()
  sentence = row['CLAIM']
  s1=[sentence]
  s1+=nltk.sent_tokenize(s2)
  #print(s1)
  sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
  X = sklearn_tfidf.fit_transform(s1)
  #print(X)
  from sklearn.metrics.pairwise import cosine_similarity
  #print(X.shape)
  s=cosine_similarity(X[0:1], X)
  #print(s.shape)
    #res=np.asarray(np.where(s >= 0.05))
    #print(s)
  p = []
  q = []
  for i in s:
    for j in i:
      p.append(j)
      q.append(j)
    #print(p)
  index_i = []
  maxElements = []
  for i in range(4):
    max_ele=0
    index = 0
    index_max = index
    for j in p:
      if(j>max_ele):
        max_ele = j
        index_max = index
      index = index+1
    index_i.append(index_max)
    p[index_max]=0.0
    maxElements.append(max_ele)
      #print(index_i)
      #print(maxElements)
      #print(s1)
  s3=""
  for i in index_i:
    if(i!=0):
    #print(q[i])
     s3+=s1[i]
  e.append(s3)
#print(len(e))
df1["EVIDENCE"]=pd.Series(e)
#print(df1)
df1.to_csv("dataset_evidence.csv", encoding="utf-8")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
949


In [0]:
import pandas as pd
import nltk
nltk.download('punkt')
df1=pd.read_csv("dataset_original.csv")
#col_names =  ['CLAIM', 'EVIDENCE', 'LABEL', 'LABEL1']
df  = pd.DataFrame()
for index,row in df1.iterrows():
  df.insert(index,"CLAIM",row['CLAIM'])
  df.insert(index,"LABEL",row['LABEL'])
  df.insert(index,"LABEL1",row['LABEL1'])
print(df)
  
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: ignored

In [0]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import re, math
from collections import Counter
import spacy
import nltk
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn 
import csv
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
import itertools
from numpy import nan

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split  

x_cosine = []
x_jaccard_sim = []
x_get_grammer = []
def get_grammer(sent1,sent2):
	#use stemmer 
  stm = PorterStemmer()
	#sent1 =  "Bheem likes eating alot."
	#sent2 = "Bheem likes eating pizza."
	#Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets
  tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
  s1 = nltk.pos_tag(nltk.word_tokenize(sent1))
  print(s1)
  s1 = dict(filter(lambda x: len(x[1])>0, map(lambda row: (row[0],wn.synsets( stm.stem(row[0]), tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() else (row[0],[]),s1)))
  s2 = nltk.pos_tag(nltk.word_tokenize(sent2))
  print(s2)
  s2 = dict(filter(lambda x: len(x[1])>0, map(lambda row: (row[0],wn.synsets( stm.stem(row[0]), tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() else (row[0],[]),s2)))
  res = {}
  for w2,gr2 in s2.items():
    for w1,gr1 in s1.items():
      tmp = pd.Series(list(map(lambda row: row[1].path_similarity(row[0]), itertools.product(gr1,gr2)))).dropna()
      if len(tmp)>0:
        res[(w1,w2)] = tmp.max()
  similarity = pd.Series(res).groupby(level=0).max().mean()
  if(similarity==None):
    similarity = 0
  return similarity


def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    score /= count
    return score
'''
WORD = re.compile(r'\w+')

def get_cosine(text1, text2):
	 vec1 = text_to_vector(text1)
	 vec2 = text_to_vector(text2)
	 intersection = set(vec1.keys()) & set(vec2.keys())
	 numerator = sum([vec1[x] * vec2[x] for x in intersection])
	 sum1 = sum([vec1[x]**2 for x in vec1.keys()])
	 sum2 = sum([vec2[x]**2 for x in vec2.keys()])
	 denominator = math.sqrt(sum1) * math.sqrt(sum2)
	 if not denominator:
	 	return 0.0
	 else:
	 	return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

text1 = 'This is our final try.'
text2 = 'We have a submission of CiCling 2019. So, this is our final try'

stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(text1)   
filtered_sentence = [w for w in word_tokens if not w in stop_words]
filtered_sentence = ""
for w in word_tokens:
	if w not in stop_words:
		filtered_sentence+=(w + " ") 
  
text1 = filtered_sentence

word_tokens = word_tokenize(text2)   
filtered_sentence = [w for w in word_tokens if not w in stop_words]
filtered_sentence = ""
for w in word_tokens:
	if w not in stop_words:
		filtered_sentence+=(w + " ")
  
text2 = filtered_sentence 



cosine = get_cosine(text1, text2)
jaccard_sim = get_jaccard_sim(text1, text2)

x_values = {}

X = []
Y = []
def value_of(label):
	if(label=='OC'):
		return 0
	else:
		if(label=='IC'):
			return 1

def decode_value(value):
	if(value==0):
		return 'OC'
	else:
		if(value==1):
			return 'IC'
#i=0
with open('dataset_evidence.csv') as csvfile:
	spamreader = csv.DictReader(csvfile)
	for row in spamreader:
		#i+=1
		#if(i==50):
		#	break
		#print(row['claim'], row['evidence'])
		Y.extend([value_of(row['LABEL1'])])
		text = row['Evidence']
		stop_words = set(stopwords.words('english')) 
		word_tokens = word_tokenize(row['CLAIM'])   
		filtered_sentence = [w for w in word_tokens if not w in stop_words]
		filtered_sentence = ""
		for w in word_tokens: 
    			if w not in stop_words: 
        			filtered_sentence+=(w + " ") 
  
		t1 = filtered_sentence 
		stop_words = set(stopwords.words('english')) 
		word_tokens = word_tokenize(str(text))   
		filtered_sentence = [w for w in word_tokens if not w in stop_words]
		filtered_sentence = ""
		for w in word_tokens: 
    			if w not in stop_words: 
        			filtered_sentence+=(w + " ") 
  
		t2 = filtered_sentence
		#x_cosine.append(get_cosine(str(text), row['CLAIM']))
		#x_jaccard_sim.append(get_jaccard_sim(str(text), row['CLAIM']))
		#x_get_grammer.append(get_grammer(str(text), row['CLAIM']))
		#x_values[get_cosine(str(text), row['CLAIM'])] = get_grammer(str(text), row['CLAIM'])
		X.extend([[get_cosine(str(text), row['CLAIM']), get_grammer(str(text),row['CLAIM'])]])
#print('X Cosine similarity: ', x_cosine)
#print('X Jaccard similarity: ', x_jaccard_sim)

#print('X grammer similarity: ', x_get_grammer)

#print(x_values)
DataElement = []
cleanedList = []
print(X)
print(Y)
for i in X:
 for j in i:
  if(str(j)!='nan'):
   DataElement.append(j)
  else:
   DataElement.append(0.0)
 cleanedList.append(DataElement)

X_train, X_test, y_train, y_test = train_test_split(cleanedList, Y, test_size = 0.40)

print(X_train)

from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)  

y_pred = svclassifier.predict(X_test)  

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 

'''
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
GaussianNB(priors=None)
print("Running sample: ")
print("CLAIM: ",text1)
print("SOURCE: ",text2)
print(decode_value(clf.predict([[get_cosine(text1, text2), get_grammer(text1,text2)]])))
'''
'''
res=[]
with open('data set 1.csv') as csvfile:
	sr=csv.DictReader(csvfile)
	for row in sr:
		file_name  = row['SOURCE'] + ".txt"
		with open(file_name, 'rb') as f:
			text = f.read()
		stop_words = set(stopwords.words('english')) 
		word_tokens = word_tokenize(row['CLAIM'])   
		filtered_sentence = [w for w in word_tokens if not w in stop_words]
		filtered_sentence = ""
		for w in word_tokens: 
    			if w not in stop_words: 
        			filtered_sentence+=(w + " ")
		t1 = filtered_sentence 
		stop_words = set(stopwords.words('english')) 
		word_tokens = word_tokenize(str(text))   
		filtered_sentence = [w for w in word_tokens if not w in stop_words]
		filtered_sentence = ""
		for w in word_tokens:
			if w not in stop_words:
				filtered_sentence+=(w + " ")
		t2 = filtered_sentence

		res.append(decode_value(clf.predict([[get_cosine(t1, t2), get_grammer(t1,t2)]])))
        #res.appends(decode_value(clf.predict([[get_cosine(text1, text2), get_jaccard_sim(text1,text2)]])))

ret=pd.Series(res)
result=pd.DataFrame()
result.insert(loc=0, column='claim', value=ret)
result.to_csv('test_result.csv',index=False)
#print('Sentence Similarity:',sentence_similarity(text1, text2))
'''

[('Moral', 'JJ'), ('Lesson', 'NNP'), ('This', 'DT'), ('lesson', 'NN'), ('teaches', 'VBZ'), ('your', 'PRP$'), ('children', 'NNS'), ('to', 'TO'), ('be', 'VB'), ('patient', 'JJ'), ('with', 'IN'), ('loved', 'JJ'), ('ones', 'NNS'), (',', ','), ('even', 'RB'), ('if', 'IN'), ('they', 'PRP'), ('are', 'VBP'), ('committing', 'VBG'), ('mistakes', 'NNS'), (',', ','), ('without', 'IN'), ('giving', 'VBG'), ('up', 'RP'), ('on', 'IN'), ('them', 'PRP'), ('.', '.')]
[('Danes', 'NNS'), ('were', 'VBD'), ('good', 'JJ'), ('persons', 'NNS'), ('.', '.')]
[('Moral', 'JJ'), ('Lesson', 'NNP'), ('This', 'DT'), ('lesson', 'NN'), ('teaches', 'VBZ'), ('your', 'PRP$'), ('children', 'NNS'), ('to', 'TO'), ('be', 'VB'), ('patient', 'JJ'), ('with', 'IN'), ('loved', 'JJ'), ('ones', 'NNS'), (',', ','), ('even', 'RB'), ('if', 'IN'), ('they', 'PRP'), ('are', 'VBP'), ('committing', 'VBG'), ('mistakes', 'NNS'), (',', ','), ('without', 'IN'), ('giving', 'VBG'), ('up', 'RP'), ('on', 'IN'), ('them', 'PRP'), ('.', '.')]
[('People'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[  0  61]
 [  0 203]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.77      1.00      0.87       203

   micro avg       0.77      0.77      0.77       264
   macro avg       0.38      0.50      0.43       264
weighted avg       0.59      0.77      0.67       264



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'\nres=[]\nwith open(\'data set 1.csv\') as csvfile:\n\tsr=csv.DictReader(csvfile)\n\tfor row in sr:\n\t\tfile_name  = row[\'SOURCE\'] + ".txt"\n\t\twith open(file_name, \'rb\') as f:\n\t\t\ttext = f.read()\n\t\tstop_words = set(stopwords.words(\'english\')) \n\t\tword_tokens = word_tokenize(row[\'CLAIM\'])   \n\t\tfiltered_sentence = [w for w in word_tokens if not w in stop_words]\n\t\tfiltered_sentence = ""\n\t\tfor w in word_tokens: \n    \t\t\tif w not in stop_words: \n        \t\t\tfiltered_sentence+=(w + " ")\n\t\tt1 = filtered_sentence \n\t\tstop_words = set(stopwords.words(\'english\')) \n\t\tword_tokens = word_tokenize(str(text))   \n\t\tfiltered_sentence = [w for w in word_tokens if not w in stop_words]\n\t\tfiltered_sentence = ""\n\t\tfor w in word_tokens:\n\t\t\tif w not in stop_words:\n\t\t\t\tfiltered_sentence+=(w + " ")\n\t\tt2 = filtered_sentence\n\n\t\tres.append(decode_value(clf.predict([[get_cosine(t1, t2), get_grammer(t1,t2)]])))\n        #res.appends(decode_value(c

In [0]:
!unzip stories.zip

Archive:  stories.zip
   creating: stories/
  inflating: stories/txt37.txt       
  inflating: stories/grammerText.txt  
  inflating: stories/Valifact_first.py  
  inflating: stories/txt3.txt        
  inflating: stories/txt1.txt        
  inflating: stories/txt6.txt        
  inflating: stories/txt2.txt        
  inflating: stories/txt5.txt        
  inflating: stories/txt4.txt        
  inflating: stories/txt20.txt       
  inflating: stories/txt12.txt       
  inflating: stories/txt21.txt       
  inflating: stories/txt17.txt       
  inflating: stories/txt14.txt       
  inflating: stories/txt16.txt       
  inflating: stories/txt18.txt       
  inflating: stories/txt27.txt       
  inflating: stories/txt19.txt       
  inflating: stories/txt25.txt       
  inflating: stories/txt7.txt        
  inflating: stories/txt38.txt       
  inflating: stories/Valifact_first_NOEvidence.py  
  inflating: stories/dataset_original.csv  
  inflating: stories/Book1.csv       
  inflating: stories